<a href="https://colab.research.google.com/github/nagamukesh/NLP-SMP/blob/Utkarsh_Shukla/NLP_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'email-classification-nlp:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F888171%2F1507821%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240701%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240701T140359Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D606c7814cf480991d03fb8e0312da16ad19612d8495193000e04b7a3f1f04375076bded675f08bebdedb6d9a1a7462e7cf3fe2ed1952ce1e2422f8232c1bc5ca0aa72f86d213cf256daafe81a3ec3a98d63f3ffced6ea29a9920d371d52aca00f58cf65515750432a21f2da1313848bd398125138dfb2f22ac31bff6ca9d21bfbeb186b420152ebfb3371d1b1aeda6c66baa2bd2dc8861de0642b2f6742118df2552a4c5140447b48c7107082f486d97e904d8050e287e260100f35cf4af2c50abeeaa25cbb5252bc1fb83da8504147ed0337aeab966b4735d3ef2415b0f9435039054966d2ae6e06e7ea72b1803105d898a9ea50788f335c4795f09e4043690'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 47917 bytes downloaded
Downloaded and uncompressed: email-classification-nlp
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-classification-nlp/SMS_train.csv
/kaggle/input/email-classification-nlp/SMS_test.csv


In [19]:
import warnings
warnings.filterwarnings("ignore")

import re
import string
from collections import Counter


import matplotlib.pyplot as plt


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix

In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# # # **Data Retrieval**

In [7]:
train_data = pd.read_csv('/kaggle/input/email-classification-nlp/SMS_train.csv',encoding='latin1')
test_data = pd.read_csv('/kaggle/input/email-classification-nlp/SMS_test.csv',encoding='latin1')
train_data.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [8]:
test_data.head()

,S. No.,Message_body,Label
0,1,"UpgrdCentre Orange customer, you may now claim...",Spam
1,2,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,3,Congrats! Nokia 3650 video camera phone is you...,Spam
3,4,URGENT! Your Mobile number has been awarded wi...,Spam
4,5,Someone has contacted our dating service and e...,Spam


In [9]:
train_data.isnull().sum()

S. No.          0
Message_body    0
Label           0
dtype: int64

In [10]:
train_data = train_data.drop('S. No.',axis=1)
test_data = test_data.drop('S. No.',axis=1)

In [11]:
print(train_data.head())
test_data.head()

                                        Message_body     Label
0                         Rofl. Its true to its name  Non-Spam
1  The guy did some bitching but I acted like i'd...  Non-Spam
2  Pity, * was in mood for that. So...any other s...  Non-Spam
3               Will ü b going to esplanade fr home?  Non-Spam
4  This is the 2nd time we have tried 2 contact u...      Spam


,Message_body,Label
0,"UpgrdCentre Orange customer, you may now claim...",Spam
1,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,Congrats! Nokia 3650 video camera phone is you...,Spam
3,URGENT! Your Mobile number has been awarded wi...,Spam
4,Someone has contacted our dating service and e...,Spam


In [12]:
train_data.columns = ['text','label']
test_data.columns = ['text','label']

In [13]:
print(train_data.head())
test_data.head()

                                                text     label
0                         Rofl. Its true to its name  Non-Spam
1  The guy did some bitching but I acted like i'd...  Non-Spam
2  Pity, * was in mood for that. So...any other s...  Non-Spam
3               Will ü b going to esplanade fr home?  Non-Spam
4  This is the 2nd time we have tried 2 contact u...      Spam


,text,label
0,"UpgrdCentre Orange customer, you may now claim...",Spam
1,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,Congrats! Nokia 3650 video camera phone is you...,Spam
3,URGENT! Your Mobile number has been awarded wi...,Spam
4,Someone has contacted our dating service and e...,Spam


In [14]:
train_data['label'] = train_data['label'].replace(['Spam','Non-Spam'],[1,0])
test_data['label'] = test_data['label'].replace(['Spam','Non-Spam'],[1,0])

In [15]:
print(train_data.head())
train_data.head()

                                                text  label
0                         Rofl. Its true to its name      0
1  The guy did some bitching but I acted like i'd...      0
2  Pity, * was in mood for that. So...any other s...      0
3               Will ü b going to esplanade fr home?      0
4  This is the 2nd time we have tried 2 contact u...      1


,text,label
0,Rofl. Its true to its name,0
1,The guy did some bitching but I acted like i'd...,0
2,"Pity, * was in mood for that. So...any other s...",0
3,Will ü b going to esplanade fr home?,0
4,This is the 2nd time we have tried 2 contact u...,1


# ## Pre-Processing

In [16]:
def clean_text(text):
    #removed special characters
    text =re.sub(r'[^a-zA-ZÀ-ú]+',' ',text)

    #lowercase the letters
    text = text.lower()

    #remove punctuations
    text = text.translate(str.maketrans('','',string.punctuation))

    #removeing whitespaces
    text = re.sub(r'\s+',' ',text).strip()
    return text

train_data['clean_text'] = train_data['text'].apply(lambda x : clean_text(x))
test_data['clean_text'] = test_data['text'].apply(lambda x : clean_text(x))


In [17]:
train_data.head()

,text,label,clean_text
0,Rofl. Its true to its name,0,rofl its true to its name
1,The guy did some bitching but I acted like i'd...,0,the guy did some bitching but i acted like i d...
2,"Pity, * was in mood for that. So...any other s...",0,pity was in mood for that so any other suggest...
3,Will ü b going to esplanade fr home?,0,will b going to esplanade fr home
4,This is the 2nd time we have tried 2 contact u...,1,this is the nd time we have tried contact u u ...


In [22]:
#Removal of stopwords
import nltk
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text.lower())
    return " ".join([token for token in tokens if token not in stop_words])

train_data['clean_text'] = train_data['clean_text'].apply(lambda x:remove_stopwords(x))
test_data['clean_text'] = test_data['clean_text'].apply(lambda x:remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
train_data.head()

,text,label,clean_text
0,Rofl. Its true to its name,0,rofl true name
1,The guy did some bitching but I acted like i'd...,0,guy bitching acted like interested buying some...
2,"Pity, * was in mood for that. So...any other s...",0,pity mood suggestions
3,Will ü b going to esplanade fr home?,0,b going esplanade fr home
4,This is the 2nd time we have tried 2 contact u...,1,nd time tried contact u u pound prize claim ea...


In [24]:
#Stemming
def normalize_text(text):
    stemmer = SnowballStemmer('english')
    normalized_text = []

    for word in text.split():
        stemmed_word = stemmer.stem(word)
        normalized_text.append(stemmed_word)

    return " ".join(normalized_text)

train_data['stemmed'] = train_data['clean_text'].apply(lambda x:normalize_text(x))
test_data['stemmed'] = test_data['clean_text'].apply(lambda x:normalize_text(x))

In [25]:
train_data.head()

,text,label,clean_text,stemmed
0,Rofl. Its true to its name,0,rofl true name,rofl true name
1,The guy did some bitching but I acted like i'd...,0,guy bitching acted like interested buying some...,guy bitch act like interest buy someth els nex...
2,"Pity, * was in mood for that. So...any other s...",0,pity mood suggestions,piti mood suggest
3,Will ü b going to esplanade fr home?,0,b going esplanade fr home,b go esplanad fr home
4,This is the 2nd time we have tried 2 contact u...,1,nd time tried contact u u pound prize claim ea...,nd time tri contact u u pound prize claim easi...


# # # Vectorization

In [26]:
# Vectorization: CountVectorizer
max_words = 10000
count_vectorizer = CountVectorizer(max_features=max_words)
count_train = count_vectorizer.fit_transform(train_data['stemmed'])
count_test = count_vectorizer.transform(test_data['stemmed'])

# Get the actual number of features from the vectorizer
input_dim_count = count_train.shape[1]

# Convert sparse matrices to dense
count_train = count_train.toarray()
count_test = count_test.toarray()

In [27]:
# Vectorization: TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=max_words)
tfidf_train = tfidf_vectorizer.fit_transform(train_data['stemmed'])
tfidf_test = tfidf_vectorizer.transform(test_data['stemmed'])

# Get the actual number of features from the vectorizer
input_dim_tfidf = tfidf_train.shape[1]


# Convert sparse matrices to dense
tfidf_train = tfidf_train.toarray()
tfidf_test = tfidf_test.toarray()

# # # Model Training

In [28]:
# Define and compile model for CountVectorizer
model_count = tf.keras.Sequential([
    Dense(16, input_shape=(input_dim_count,), activation='relu'),
    Dense(1, activation='sigmoid')
])
model_count.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model with CountVectorizer
history_count = model_count.fit(count_train, train_data['label'], epochs=10, batch_size=64, validation_data=(count_test, test_data['label']), verbose=2)

# Evaluate model with CountVectorizer
loss_count, accuracy_count = model_count.evaluate(count_test, test_data['label'], verbose=0)
print(f"CountVectorizer Test Accuracy: {accuracy_count*100:.2f}%")

Epoch 1/10
15/15 - 3s - loss: 0.6497 - accuracy: 0.8401 - val_loss: 0.6764 - val_accuracy: 0.5280 - 3s/epoch - 173ms/step
Epoch 2/10
15/15 - 0s - loss: 0.5648 - accuracy: 0.9258 - val_loss: 0.6444 - val_accuracy: 0.6560 - 104ms/epoch - 7ms/step
Epoch 3/10
15/15 - 0s - loss: 0.4773 - accuracy: 0.9446 - val_loss: 0.6123 - val_accuracy: 0.7040 - 116ms/epoch - 8ms/step
Epoch 4/10
15/15 - 0s - loss: 0.3953 - accuracy: 0.9645 - val_loss: 0.5729 - val_accuracy: 0.7360 - 131ms/epoch - 9ms/step
Epoch 5/10
15/15 - 0s - loss: 0.3257 - accuracy: 0.9749 - val_loss: 0.5265 - val_accuracy: 0.7840 - 143ms/epoch - 10ms/step
Epoch 6/10
15/15 - 0s - loss: 0.2687 - accuracy: 0.9833 - val_loss: 0.4881 - val_accuracy: 0.8160 - 116ms/epoch - 8ms/step
Epoch 7/10
15/15 - 0s - loss: 0.2231 - accuracy: 0.9896 - val_loss: 0.4538 - val_accuracy: 0.8320 - 128ms/epoch - 9ms/step
Epoch 8/10
15/15 - 0s - loss: 0.1868 - accuracy: 0.9927 - val_loss: 0.4284 - val_accuracy: 0.8400 - 121ms/epoch - 8ms/step
Epoch 9/10
15/15

In [29]:
# Define and compile model for TF-IDF Vectorizer
model_tfidf = tf.keras.Sequential([
    Dense(16, input_shape=(input_dim_tfidf,), activation='relu'),
    Dense(1, activation='sigmoid')
])
model_tfidf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model with TF-IDF Vectorizer
history_tfidf = model_tfidf.fit(tfidf_train, train_data['label'], epochs=10, batch_size=64, validation_data=(tfidf_test, test_data['label']), verbose=2)

# Evaluate model with TF-IDF Vectorizer
loss_tfidf, accuracy_tfidf = model_tfidf.evaluate(tfidf_test, test_data['label'], verbose=0)
print(f"TF-IDF Test Accuracy: {accuracy_tfidf*100:.2f}%")

Epoch 1/10
15/15 - 1s - loss: 0.6699 - accuracy: 0.8464 - val_loss: 0.6931 - val_accuracy: 0.4080 - 811ms/epoch - 54ms/step
Epoch 2/10
15/15 - 0s - loss: 0.6216 - accuracy: 0.8788 - val_loss: 0.6866 - val_accuracy: 0.4000 - 79ms/epoch - 5ms/step
Epoch 3/10
15/15 - 0s - loss: 0.5642 - accuracy: 0.8830 - val_loss: 0.6774 - val_accuracy: 0.4320 - 67ms/epoch - 4ms/step
Epoch 4/10
15/15 - 0s - loss: 0.5057 - accuracy: 0.8924 - val_loss: 0.6664 - val_accuracy: 0.5200 - 88ms/epoch - 6ms/step
Epoch 5/10
15/15 - 0s - loss: 0.4499 - accuracy: 0.9070 - val_loss: 0.6522 - val_accuracy: 0.5280 - 66ms/epoch - 4ms/step
Epoch 6/10
15/15 - 0s - loss: 0.3985 - accuracy: 0.9164 - val_loss: 0.6377 - val_accuracy: 0.5680 - 82ms/epoch - 5ms/step
Epoch 7/10
15/15 - 0s - loss: 0.3522 - accuracy: 0.9321 - val_loss: 0.6200 - val_accuracy: 0.6160 - 87ms/epoch - 6ms/step
Epoch 8/10
15/15 - 0s - loss: 0.3113 - accuracy: 0.9436 - val_loss: 0.6032 - val_accuracy: 0.6480 - 94ms/epoch - 6ms/step
Epoch 9/10
15/15 - 0s 

In [30]:
# Confusion Matrix for CountVectorizer
predictions_count = (model_count.predict(count_test) > 0.5).astype("int32")
cm_count = confusion_matrix(test_data['label'], predictions_count)
print("CountVectorizer Confusion Matrix:")
print(cm_count)

4/4 [==============================] - 0s 3ms/step
CountVectorizer Confusion Matrix:
[[49  0]
 [18 58]]


In [31]:
# Confusion Matrix for TF-IDF Vectorizer
predictions_tfidf = (model_tfidf.predict(tfidf_test) > 0.5).astype("int32")
cm_tfidf = confusion_matrix(test_data['label'], predictions_tfidf)
print("TF-IDF Confusion Matrix:")
print(cm_tfidf)

4/4 [==============================] - 0s 3ms/step
TF-IDF Confusion Matrix:
[[49  0]
 [39 37]]


In [32]:
# train_data.head()

In [33]:
# import collections
# words = []
# for text in train_data['lemmatized']:
#     words.extend(text.split())
# word_count = collections.Counter(words)
# top_words = dict(word_count.most_common(10))

# #Figure Size
# plt.figure(figsize = (10,6))

# #Barplot
# plt.bar(range(len(top_words)), list(top_words.values()), align = 'center')

# plt.xticks(range(len(top_words)),list(top_words.keys()))

# plt.grid(alpha=0.5)

# plt.title('Top 10 most used words',fontsize=18)
# plt.xlabel('Words')
# plt.ylabel('Frequency')

In [ ]:
# #max_words to be considered in vocabulary
# max_words =10000

# #max_tokens to be consider in a sequence
# max_len =200

# tokenizer = Tokenizer(num_words = max_words)

# tokenizer.fit_on_texts(train_data['lemmatized'])

# sequences_train = tokenizer.texts_to_sequences(train_data['lemmatized'])
# sequences_val = tokenizer.texts_to_sequences(test_data['lemmatized'])

# word_index = tokenizer.word_index

In [ ]:
# #padding
# data_train = pad_sequences(sequences_train,maxlen =max_len)
# data_val = pad_sequences(sequences_val,maxlen=max_len)

In [ ]:
# # Create the model
# model = tf.keras.Sequential()
# model.add(Embedding(max_words, 16, input_length = max_len))
# model.add(GlobalAveragePooling1D())
# model.add(Dense(1, activation = 'sigmoid'))

# # Compile the model
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [34]:
# history = model.fit(data_train, train_data['label'], epochs = 100, batch_size = 64, validation_data = (data_val, test_data['label']))

In [ ]:
# model.summary()

In [ ]:
# loss, accuracy = model.evaluate(data_val, test_data['label'], verbose = 0)
# print('Accuracy: %f' % (accuracy*100))